<a href="https://colab.research.google.com/github/maulanaakbardj/notebook_TG/blob/main/TG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create and Setup Graph

In [ ]:
#Setup
!pip install pyTigerGraph

In [ ]:
import pyTigerGraph as tg
conn = tg.TigerGraphConnection(host="https://#your_subdomain#.i.tgcloud.io", password="#your_pass")

In [ ]:
print(conn.gsql('''
  CREATE VERTEX Person (PRIMARY_ID id STRING, name STRING, email STRING, username STRING, created_at DATETIME) WITH primary_id_as_attribute="true"
  CREATE VERTEX Post (PRIMARY_ID id STRING, content STRING, posted_date DATETIME, deleted BOOL) WITH primary_id_as_attribute="true"
  CREATE VERTEX Hastag (PRIMARY_ID tag STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX Message (PRIMARY_ID id STRING, subject STRING, body STRING) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE posted (From Person, To Post, post_date DATETIME) WITH REVERSE_EDGE="reverse_posted"
  CREATE DIRECTED EDGE liked (From Person, To Post, like_date DATETIME) WITH REVERSE_EDGE="reverse_liked"
  CREATE DIRECTED EDGE has_tag (From Post, To Hastag) WITH REVERSE_EDGE="reverse_likedhas_tag"
  CREATE DIRECTED EDGE sent_message (From Person, To Message, to_person STRING, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_message"
  CREATE DIRECTED EDGE received_message (From Message, To Person, from_person STRING, receive_date DATETIME, opened_date DATETIME) WITH REVERSE_EDGE="reverse_received_message"
''', options=[]))

Successfully created vertex types: [Person].
Successfully created vertex types: [Post].
Successfully created vertex types: [Hastag].
Successfully created vertex types: [Message].
Successfully created edge types: [posted].
Successfully created reverse edge types: [reverse_posted].
Successfully created edge types: [liked].
Successfully created reverse edge types: [reverse_liked].
Successfully created edge types: [has_tag].
Successfully created reverse edge types: [reverse_likedhas_tag].
Successfully created edge types: [sent_message].
Successfully created reverse edge types: [reverse_sent_message].
Successfully created edge types: [received_message].
Successfully created reverse edge types: [reverse_received_message].


In [ ]:
print(conn.gsql('''CREATE GRAPH MyGraph_1(Person, Post, Hastag, Message, posted, liked, has_tag, sent_message, received_message)''', options=[]))

The graph MyGraph_1 is created.


In [ ]:
conn.graphname = "MyGraph_1"
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)
print(token)

('trnrnbiidnd9kc05d1hduoedr6hrktp3', 1647035669, '2022-03-11 21:54:29')


In [ ]:
!git clone https://github.com/maulanaakbardj/TigerGraph-101

Cloning into 'TigerGraph-101'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [28]:
!head -n 2 /content/TigerGraph-101/posts.csv

id,content,posted_date,by_user,deleted,hashtag_1,hashtag_2,hashtag_3,hashtag_4
1,"Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl. Aenean lectus. Pellentesque eget nunc. Donec quis orci eget orci vehicula condimentum. Curabitur in libero ut massa volutpat convallis. Morbi odio odio, elementum eu, interdum eu, tincidunt in, leo.",2019-08-04 20:43:08,69,False,Compatible,Organized,,workforce


In [ ]:
results= conn.gsql('''
  USE GRAPH MyGraph_1
  BEGIN
  CREATE LOADING JOB load_posts FOR GRAPH MyGraph_1 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Post VALUES($0, $1, $2, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hastag VALUES($5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hastag VALUES($6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hastag VALUES($7) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO VERTEX Hastag VALUES($8) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE posted VALUES($3, $0, $2) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $7) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE has_tag VALUES($0, $8) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
''', options=[])
print(results)

Using graph 'MyGraph_1'
Successfully created loading jobs: [load_posts].


In [ ]:
!head -n 2 /content/TigerGraph-101/likes.csv

id,by_user,liked_post,liked_date
1,65,798,2019-01-02 08:11:14


In [ ]:
results= conn.gsql('''
  USE GRAPH MyGraph_1
  BEGIN
  CREATE LOADING JOB load_likes FOR GRAPH MyGraph_1 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE liked VALUES($1, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
''', options=[])
print(results)

Using graph 'MyGraph_1'
Successfully created loading jobs: [load_likes].


In [ ]:
!head -n 2 /content/TigerGraph-101/messages.csv

id,body,subject,by_user,to_user,send_date,read_date
1,"Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula. Suspendisse ornare consequat lectus. In est risus, auctor sed, tristique in, tempus sit amet, sem. Fusce consequat. Nulla nisl.",Nulla facilisi.,22,62,2019-05-01 03:40:51,2019-06-10 03:40:51


In [ ]:
results= conn.gsql('''
  USE GRAPH MyGraph_1
  BEGIN
  CREATE LOADING JOB load_messages FOR GRAPH MyGraph_1 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Message VALUES($0, $2, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE sent_message VALUES($3, $0, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE received_message VALUES($0, $4, $3, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
''', options=[])
print(results)

Using graph 'MyGraph_1'
Successfully created loading jobs: [load_messages].


In [ ]:
!head -n 2 /content/TigerGraph-101/users.csv

id,email,username,name,join_date
1,mgeistmann0@accuweather.com,mgeistmann0,Marvin Geistmann,2018-09-30 07:31:34


In [ ]:
results=conn.gsql('''
  USE GRAPH MyGraph_1
  BEGIN
  CREATE LOADING JOB load_people FOR GRAPH MyGraph_1 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Person VALUES($0, $3, $1, $2, $4) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
''', options=[])
print(results)

Using graph 'MyGraph_1'
Successfully created loading jobs: [load_people].


In [31]:
import json

In [32]:
#Load the posts file with the 'load_posts' job
posts_file='/content/TigerGraph-101/posts.csv'
results=conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_posts')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Post",
          "validObject": 1000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:posted_date"
          ],
          "invalidAttributeLinesData": [
            "id,content,posted_date,by_user,deleted,hashtag_1,hashtag_2,hashtag_3,hashtag_4\r\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "Hastag",
          "validObject": 1001,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "inc

In [33]:
#Load the posts file with the 'load_likes' job
likes_file='/content/TigerGraph-101/likes.csv'
results=conn.uploadFile(likes_file, fileTag='MyDataSource', jobName='load_likes')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 5001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "liked",
          "validObject": 5000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:like_date"
          ],
          "invalidAttributeLinesData": [
            "id,by_user,liked_post,liked_date\r\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [34]:
#Load the posts file with the 'load_messages' job
messages_file='/content/TigerGraph-101/messages.csv'
results=conn.uploadFile(messages_file, fileTag='MyDataSource', jobName='load_messages')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1001,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Message",
          "validObject": 1001,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "sent_message",
          "validObject": 1000,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:sent_date"
          ],
          "invalidAttributeLinesData": [
            "id,body,subject,by_user,to_user,send_date,read_date\r\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          

In [35]:
#Load the posts file with the 'load_people' job
people_file='/content/TigerGraph-101/users.csv'
results=conn.uploadFile(people_file, fileTag='MyDataSource', jobName='load_people')
print(json.dumps(results,indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 101,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Person",
          "validObject": 100,
          "noIdFound": 0,
          "invalidAttribute": 1,
          "invalidAttributeLines": [
            "1:created_at"
          ],
          "invalidAttributeLinesData": [
            "id,email,username,name,join_date\r\n"
          ],
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


# Get Vertex and Edge Schema

In [36]:
results = conn.getVertexTypes()
print(f"veticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

veticies: ['Person', 'Post', 'Hastag', 'Message']
Edges: ['posted', 'liked', 'has_tag', 'sent_message', 'received_message']


In [54]:
print("Vertex count")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------------------")
print("Edge count")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex count
There are 100 Person vertices in the graph
There are 1001 Post vertices in the graph
There are 308 Hastag vertices in the graph
There are 1001 Message vertices in the graph
--------------------------
Edge count
There are 1000 posted edges in the graph
There are 4869 liked edges in the graph
There are 2567 has_tag edges in the graph
There are 1000 sent_message edges in the graph
There are 1000 received_message edges in the graph


# Getting Vertex

In [57]:
import pprint as pp
results = conn.getVerticesById("Person", "30")
pp.pprint(results)

[{'attributes': {'created_at': '2018-12-23 08:52:51',
                 'email': 'gmattint@dion.ne.jp',
                 'id': '30',
                 'name': 'Garrot Mattin',
                 'username': 'gmattint'},
  'v_id': '30',
  'v_type': 'Person'}]


In [59]:
reprint = conn.getVerticesById("Post", ["300","400"])
pp.pprint(reprint)

[{'attributes': {'content': 'In congue. Etiam justo.',
                 'deleted': False,
                 'id': '300',
                 'posted_date': '2020-10-15 11:25:30'},
  'v_id': '300',
  'v_type': 'Post'},
 {'attributes': {'content': 'Nulla neque libero, convallis eget, eleifend '
                            'luctus, ultricies eu, nibh. Quisque id justo sit '
                            'amet sapien dignissim vestibulum. Vestibulum ante '
                            'ipsum primis in faucibus orci luctus et ultrices '
                            'posuere cubilia Curae; Nulla dapibus dolor vel '
                            'est. Donec odio justo, sollicitudin ut, suscipit '
                            'a, feugiat et, eros.',
                 'deleted': False,
                 'id': '400',
                 'posted_date': '2020-03-16 11:19:30'},
  'v_id': '400',
  'v_type': 'Post'}]


# Count Edges Connected to a vertex

In [61]:
results = conn.getEdgeCountFrom("Person", "30")
pp.pprint(results)

{'has_tag': 0,
 'liked': 47,
 'posted': 10,
 'received_message': 0,
 'reverse_liked': 0,
 'reverse_likedhas_tag': 0,
 'reverse_posted': 0,
 'reverse_received_message': 5,
 'reverse_sent_message': 0,
 'sent_message': 8}


# Show all Edges Connected to a vertex

In [62]:
results = conn.getEdges("Person", "30")
pp.pprint(results)

[{'attributes': {'post_date': '2020-08-02 04:36:05'},
  'directed': True,
  'e_type': 'posted',
  'from_id': '30',
  'from_type': 'Person',
  'to_id': '246',
  'to_type': 'Post'},
 {'attributes': {'post_date': '2019-02-16 02:19:16'},
  'directed': True,
  'e_type': 'posted',
  'from_id': '30',
  'from_type': 'Person',
  'to_id': '998',
  'to_type': 'Post'},
 {'attributes': {'post_date': '2020-03-31 14:49:25'},
  'directed': True,
  'e_type': 'posted',
  'from_id': '30',
  'from_type': 'Person',
  'to_id': '652',
  'to_type': 'Post'},
 {'attributes': {'post_date': '2019-04-05 20:24:56'},
  'directed': True,
  'e_type': 'posted',
  'from_id': '30',
  'from_type': 'Person',
  'to_id': '766',
  'to_type': 'Post'},
 {'attributes': {'post_date': '2020-03-16 11:19:30'},
  'directed': True,
  'e_type': 'posted',
  'from_id': '30',
  'from_type': 'Person',
  'to_id': '400',
  'to_type': 'Post'},
 {'attributes': {'post_date': '2021-06-19 14:36:07'},
  'directed': True,
  'e_type': 'posted',
  'f

# Dataframe

In [63]:
df1 = conn.getVertexDataframe("Hastag")
print(df1)

                v_id              tag
0       User-centric     User-centric
1        even-keeled      even-keeled
2         methodical       methodical
3          Versatile        Versatile
4          workforce        workforce
..               ...              ...
303  disintermediate  disintermediate
304   infrastructure   infrastructure
305   implementation   implementation
306         intranet         intranet
307  standardization  standardization

[308 rows x 2 columns]


In [64]:
df2 = conn.getVertexDataframeById("Post","45")
print(df2)

  v_id  id  ...          posted_date deleted
0   45  45  ...  2019-05-30 13:14:33   False

[1 rows x 5 columns]


In [65]:
df3 = conn.getVertexDataframeById("Post",["45","344"])
print(df3)

  v_id   id  ...          posted_date deleted
0   45   45  ...  2019-05-30 13:14:33   False
1  344  344  ...  2019-02-10 11:18:04   False

[2 rows x 5 columns]


In [68]:
df4 = conn.getEdgesDataframe("Post", "344")
print(df4)

  from_type from_id  ...            post_date            like_date
0      Post     344  ...  2019-02-10 11:18:04                  NaN
1      Post     344  ...                  NaN  2021-05-08 21:31:39
2      Post     344  ...                  NaN  2018-11-01 23:29:51
3      Post     344  ...                  NaN  2018-11-02 05:14:36
4      Post     344  ...                  NaN  2018-12-30 16:54:19
5      Post     344  ...                  NaN                  NaN
6      Post     344  ...                  NaN                  NaN

[7 rows x 6 columns]


In [70]:
df5= conn.getEdgesDataframe("Post", "344", limit=3)
print(df5)

  from_type from_id to_type to_id            post_date            like_date
0      Post     344  Person    92  2019-02-10 11:18:04                  NaN
1      Post     344  Person    69                  NaN  2021-05-08 21:31:39
2      Post     344  Person    86                  NaN  2018-11-01 23:29:51


# Queries

All Hashtags from all Posts from Input User 

In [75]:
results = conn.gsql('''
  USE GRAPH MyGraph_1
  CREATE QUERY hashtags_from_person(VERTEX<Person> inPer) FOR GRAPH MyGraph_1 SYNTAX v2{
  seed = {inPer};
  tag= SELECT t FROM seed:s - (posted>) - Post:p - (has_tag>) - Hastag:t;
  PRINT tag;
  }
'''    
)
pp.pprint(results)

"Using graph 'MyGraph_1'\nSuccessfully created queries: [hashtags_from_person]."


In [76]:
conn.gsql(''' 
  INSTALL QUERY hashtags_from_person
''', options=[])

'Start installing queries, about 1 minute ...\nhashtags_from_person query: curl -X GET \'https://127.0.0.1:9000/query/MyGraph_1/hashtags_from_person?inPer=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [82]:
results = conn.runInstalledQuery("hashtags_from_person", params={"inPer":"30"})
pp.pprint(results) 

[{'tag': [{'attributes': {'tag': 'Fundamental'},
           'v_id': 'Fundamental',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'time-frame'},
           'v_id': 'time-frame',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'full-range'},
           'v_id': 'full-range',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'Implemented'},
           'v_id': 'Implemented',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'bottom-line'},
           'v_id': 'bottom-line',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'Persistent'},
           'v_id': 'Persistent',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'definition'},
           'v_id': 'definition',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'content-based'},
           'v_id': 'content-based',
           'v_type': 'Hastag'},
          {'attributes': {'tag': 'zero administration'},
           'v_id': 'zero ad